[View in Colaboratory](https://colab.research.google.com/github/gauravbansal98/Mnist-dataset-using-rnn-1-06-2018/blob/master/mnist_dataset_using_rnn.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("drive/mnist dataset using normal rnn", one_hot = True)

In [0]:
os.chdir('/content')
!git clone https://github.com/mixuala/colab_utils
import colab_utils.tboard
ROOT = %pwd
print(ROOT)
LOG_DIR = os.path.join(ROOT, 'gaurav')
print(LOG_DIR)
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

In [0]:
from tensorflow.python.ops import rnn, rnn_cell
tf.reset_default_graph()

In [0]:
rnn_size = 128
no_of_nodes_in_output_layer = 10
chunk_size = 28
no_of_chunk = 28
batch_size = 128
hm_epochs = 20

In [0]:
x = tf.placeholder(dtype = tf.float32, shape = [None, no_of_chunk, chunk_size], name = "input_labels")
y = tf.placeholder(dtype = tf.float32, shape = [None, None], name = "output_labels")

In [0]:
weights = { 'weights' : tf.Variable(tf.random_normal([rnn_size, no_of_nodes_in_output_layer], name = 'weights')),
           'biases' : tf.Variable(tf.random_normal([1, no_of_nodes_in_output_layer], name = 'biases')) }

In [0]:
def recurrent_neural_network(x):
  with tf.name_scope('Neural_Network'):
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, no_of_chunk, 0)
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size, name = 'LSTM_CELL')
    outputs, state = tf.nn.static_rnn(cell, x, dtype = tf.float32)

    output = tf.add(tf.matmul(outputs[-1], weights['weights']), weights['biases'])
    return output

In [0]:
def train_neural_network(x):

  prediction = recurrent_neural_network(x)
  
  with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
  summary = tf.summary.scalar('total_cost', cost)
  
  with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)
  
  merged = tf.summary.merge_all()
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./gaurav',sess.graph)
    k = 0
    for i in range(hm_epochs):
      epoch_loss = 0
      for j in range(int(mnist.train.num_examples/batch_size)):
        epoch_x, epoch_y = mnist.train.next_batch(batch_size)
        epoch_x = epoch_x.reshape((batch_size,no_of_chunk,chunk_size))
        o, c, m = sess.run([optimizer, cost, merged], feed_dict = {x : epoch_x, y : epoch_y})
        epoch_loss += c
        train_writer.add_summary(m, k)
        k += 1
      print('epoch' , i, 'completed out of ', hm_epochs, 'loss ', epoch_loss)
      
      
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
  
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
  
    print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape(-1,no_of_chunk,chunk_size), y:mnist.test.labels})) 

In [91]:
train_neural_network(x)

epoch 0 completed out of  20 loss  203.51496625691652
epoch 1 completed out of  20 loss  58.848411567509174
epoch 2 completed out of  20 loss  39.99966784613207
epoch 3 completed out of  20 loss  29.94585596770048
epoch 4 completed out of  20 loss  25.270001084078103
epoch 5 completed out of  20 loss  20.602786916308105
epoch 6 completed out of  20 loss  18.362319762585685
epoch 7 completed out of  20 loss  15.69905512954574
epoch 8 completed out of  20 loss  13.8019198164111
epoch 9 completed out of  20 loss  13.79600096627837
epoch 10 completed out of  20 loss  10.316648285952397
epoch 11 completed out of  20 loss  9.803733514680061
epoch 12 completed out of  20 loss  9.399962866213173
epoch 13 completed out of  20 loss  8.765975842921762
epoch 14 completed out of  20 loss  7.35447836513049
epoch 15 completed out of  20 loss  7.398822996459785
epoch 16 completed out of  20 loss  6.171237048285548
epoch 17 completed out of  20 loss  6.797430321836146
epoch 18 completed out of  20 loss